# Introduction to the S_AOI_Ar Combiner

Welcome to this Jupyter Notebook dedicated to combining feature classes in the coastal and riverine geo databases into a single feature class with a WGS 84 spatial reference. In this notebook, we will merge the newly converted feature classes and store them in their respective Geo Databases.

### Objectives
- **Merge Feature Classes**: The main goal is to combine the converted feature classes from the Coastal and Riverine datasets into a single feature class with a WGS 84 spatial reference.
- **Organize Data**: By merging the feature classes, we create standardized datasets for spatial analysis and visualization purposes.

### Prerequisites
Before proceeding, ensure that you have the necessary libraries installed in your Python environment:
- `tkinter` for creating GUI dialogs
- `arcpy` for geospatial analysis
- `pandas` for data manipulation
- `os` and `glob` for file handling

Let’s get started by loading our libraries and running the code to combine the feature classes! Remember to select the cell and press `Ctrl + Enter` to execute the code.

In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import arcpy
import pandas
import os
import glob

<div style="height: 30px;"></div>

### Step 2: Geo Database Folder Selection
Now we need to confirm the path to the Coastal and Riverine geodatabases.  This Python code snippet opens a folder selection dialog to specify the path to a Geo Database folder, select the folder that contains them, it should be named `FEMA-S-AOI-Ar-Processor`. Confirm the selected folder with a message box.

1. **Run the Code Cell Below**: Execute the cell provided to open a file dialog that will allow you to browse your system.
2. **Navigate to the Geo Datbase Folder**: In the dialog, locate and select the folder that contains your FEMA floodplain data files.
3. **Confirm Your Selection**: Once you have selected the folder, click "OK" to confirm your choice.

In [ ]:
# Open a folder selection dialog
gdb_folder_path = filedialog.askdirectory(title="Select Geo Database Folder")
confirm = messagebox.askyesno("Confirm Folder", f"You selected:\n\n{gdb_folder_path}\n\nIs this correct?")

# Specify the Coastal and Riverine GeoDatabases
cst_gdb = os.path.join(gdb_folder_path, 'Coastal.gdb')
riv_gdb = os.path.join(gdb_folder_path, 'Riverine.gdb')


# Produce Warning if Geo Databases Dont Exist
if not arcpy.Exists(cst_gdb):
    messagebox.showwarning("Warning", "Coastal Geo Database Does Not Exist.\n\nPlease Create with Setup before Continuing.")
    
# Produce Warning if Geo Databases Dont Exist
if not arcpy.Exists(riv_gdb):
    messagebox.showwarning("Warning", "Riverine Geo Database Does Not Exist.\n\nPlease Create with Setup before Continuing.")

<div style="height: 30px;"></div>

### Step 3: Process Coastal and Riverine Feature Classes and Merge

In this step, we will iterate through the list of feature classes in the coastal and riverine geo databases, merge them into respective single feature classes. Here's a detailed explanation of how to execute this process for both coastal and riverine data:

1. **Setting Up Environment**:
    - Start by ensuring that the arcpy library is correctly installed in your Python environment for geospatial analysis.
    - Determine if you want to prevent the output feature classes from being added to the map by setting `arcpy.env.addOutputsToMap = True`.

2. **Specifying Workspace for Coastal Data**:
    - Set the workspace (`cst_gdb`) variable to the path where the coastal geo database is located.

3. **Processing Coastal Feature Classes**:
    - Initialize counters for the coastal processes, such as total_counties, coastal_succeed, coastal_fail, and coastal_fail_list.
    - Assign a name (`cst_combined_name`) for the combined feature class for coastal data.
    - Obtain a list of feature classes in the coastal geo database using `arcpy.ListFeatureClasses()`.
    - Iterate through each coastal feature class, skipping the empty feature class, and attempt to merge them into the combined feature class.
    - Update success and failure counts, print messages for successful merges or failures.

4. **Specifying Workspace for Riverine Data**:
    - Set the workspace (`riv_gdb`) variable to the path where the riverine geo database is located.

5. **Processing Riverine Feature Classes**:
    - Initialize counters for the riverine processes, such as total_counties, riverine_succeed, riverine_fail, and riverine_fail_list.
    - Assign a name (`riv_combined_name`) for the combined feature class for riverine data.
    - Obtain a list of feature classes in the riverine geo database using `arcpy.ListFeatureClasses()`.
    - Iterate through each riverine feature class, skipping the empty feature class, and attempt to merge them into the combined feature class.
    - Update success and failure counts, print messages for successful merges or failures.

6. **Printing Summary Reports**:
    - After processing all feature classes for both coastal and riverine data, print summary reports showing the total number of feature classes processed, successful merges, and failed merges for each dataset.

7. **Displaying List of Failures**:
    - If there are any failed merges for coastal or riverine data, list them explicitly with the feature class name and reason for failure.

Remember to run this code in a Python environment with arcpy installed and verified workspace variables set correctly. By the end of this process, the newly combined feature classes for coastal and riverine data will be stored in their respective geo databases for further analysis and visualization.

#### Step 3A: Merge Coastal Files

In [ ]:
#Prevent from being added to the map
arcpy.env.addOutputsToMap = True

# Step 1: Grab the first feature class in the CoastalConverted folder
arcpy.env.workspace = cst_gdb

#Set Counters for Processes 
total_counties = 0
coastal_succeed = 0
coastal_fail = 0
coastal_fail_list = []

#Create Coastal Combined Feature Class Name
cst_combined_name = "Coastal_Combined_S_AOI_Ar_WGS84"

#Find List of Feature Classes
feature_classes = arcpy.ListFeatureClasses()


# Step 4: Cycle through each feature class in the geodatabase and merge them into the empty feature class one at a time
for fc in feature_classes:
    if fc != cst_combined_name:  # Skip the empty feature class

        try:
            arcpy.management.Append(os.path.join(cst_gdb, fc), cst_combined_name)
            coastal_succeed += 1
            total_counties += 1
            print(f"Merged {fc} into {cst_combined_name}.")


        except Exception as e:
            error = [fc, "Failed to Merge", e]
            coastal_fail_list.append(error)
            coastal_fail += 1
            total_counties += 1
            print(f"Failed to Merge {fc} into {cst_combined_name}.")


print("")
print("")

# Print summary
print("Summary Report")
print("=================")
print(f"Total Counties: {total_counties}")
print(f"  Successfully Processed: {coastal_succeed}")
print(f"  Failed to Process: {coastal_fail}")

print("")
print("")

#Print List of Failures if Present
if len(coastal_fail_list) > 0:
    print("Coastal Failed Conversions")

    for item in coastal_fail_list:
        print(f"  {item}")

#### Step 3B: Merge Riverine Files

In [ ]:
#Prevent from being added to the map
arcpy.env.addOutputsToMap = True

# Step 1: Grab the first feature class in the CoastalConverted folder
arcpy.env.workspace = riv_gdb

#Set Counters for Processes 
total_counties = 0
riverine_succeed = 0
riverine_fail = 0
riverine_fail_list = []


#Create Coastal Combined Feature Class Name
riv_combined_name = "Riverine_Combined_S_AOI_Ar_WGS84"

#Find List of Feature Classes
feature_classes = arcpy.ListFeatureClasses()


# Step 4: Cycle through each feature class in the geodatabase and merge them into the empty feature class one at a time
for fc in feature_classes:
    if fc != riv_combined_name:  # Skip the empty feature class

        try:
            arcpy.management.Append(os.path.join(riv_gdb, fc), riv_combined_name)
            riverine_succeed += 1
            total_counties += 1
            print(f"Merged {fc} into {riv_combined_name}.")


        except Exception as e:
            error = [fc, "Failed to Merge", e]
            riverine_fail_list.append(error)
            riverine_fail += 1
            total_counties += 1
            print(f"Failed to Merge {fc} into {riv_combined_name}.")


print("")
print("")

# Print summary
print("Summary Report")
print("=================")
print(f"Total Counties: {total_counties}")
print(f"  Successfully Processed: {riverine_succeed}")
print(f"  Failed to Process: {riverine_fail}")

print("")
print("")

#Print List of Failures if Present
if len(riverine_fail_list) > 0:
    print("Coastal Failed Conversions")

    for item in riverine_fail_list:
        print(f"  {item}")